Tatiana MAIA FERREIRA
Nicolas ROUSSEAU

Bibliographie : 
- https://numbersandshapes.net/post/fitting_sir_to_data_in_python/
- https://www.lewuathe.com/covid-19-dynamics-with-sir-model.html

Le modèle SIR est un modèle compartimental qui permet de modéliser la propagation d'un au cours du temps. On sépare l'ensemble de la population en 3 catégories : les Susceptibles, les Infectés et les Rétablis (et immunisés contre la maladie) qui composent l'ensemble de la population du pays (N).
Pour estimer le modèle, on se base sur le taux de contact moyen dans la population ainsi que l'inverse de la période infectieuse moyenne.
On part de l'hypothèse que la population est fixe (pas de morts ni de naissances).
On suit l'évolution des courbes des 3 classes à travers le temps qui au fur et à mesure se croisent les unes avec les autres. Au début les infectés sont plus nombreux que les rétablis et croissent plus fortement. À la fin, lorsque l'épidémie s'est dissipé, le nombre de rétablis capte la plupart des individus et à dépasser le nombre de personnes infectées.

Il existe des variantes au modèle SIR. On peut notamment évoquer le modèle SIS ajoute le fait qu'un individu est initialement sain (S), il peut par la suite devenir infecté (I) puis être guéri (S). Si on retire l'hypothèse de la possibilité de guérison, alors il s'agirait d'un modèle SI.

On peut également évoquer le modèle de diffusion de Bass. À la différence des 3 premiers évoqués précédemment, ce modèle a été dévéloppé initialement pour étudier les phénomènes de diffusion de produits et services, alors que les précédent sont principalement utilisés dans le domaine de la médecine. Ce modèle caractérise le développement du phénomène à travers le temps et décrit la forme "left-skewed" de la propagation : la phase d'expansion est rapide avec arrivée à un pic puis retombée.

In [1]:
import datetime
import os
import yaml

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Lecture du fichier d'environnement
ENV_FILE = '../env.yaml'
with open(ENV_FILE) as f:
    params = yaml.load(f) #, Loader=yaml.FullLoader)

# Initialisation des chemins vers les fichiers
ROOT_DIR = os.path.dirname(os.path.abspath(ENV_FILE))
DATA_FILE = os.path.join(ROOT_DIR,
                         params['directories']['processed'],
                         params['files']['all_data'])

# Lecture du fichier de données
epidemie_df = (pd.read_csv(DATA_FILE, parse_dates=['Last Update'])
               .assign(day=lambda _df: _df['Last Update'].dt.date)
               .drop_duplicates(subset=['Country/Region', 'Province/State', 'day'])
               [lambda df: df['day'] <= datetime.date(2020, 3, 12)]
              )

<ipython-input-2-165f8f9dfe66>:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  params = yaml.load(f) #, Loader=yaml.FullLoader)


In [3]:
# Définition d'une fonction qui récupère un pays et en fait un subset du dataframe initiale 
def get_country(self, country):
    return (epidemie_df[epidemie_df['Country/Region'] == country]
            .groupby(['Country/Region', 'day'])
            .agg({'Confirmed': 'sum', 'Deaths': 'sum', 'Recovered': 'sum'})
            .reset_index()
           )

# Monkey Patch pd.DataFrame
pd.DataFrame.get_country = get_country

In [5]:
spain_df = get_country(epidemie_df, "Spain")
spain_df.head()

,Country/Region,day,Confirmed,Deaths,Recovered
0,Spain,2020-02-01,1.0,0.0,0.0
1,Spain,2020-02-09,2.0,0.0,0.0
2,Spain,2020-02-15,2.0,0.0,2.0
3,Spain,2020-02-25,6.0,0.0,2.0
4,Spain,2020-02-26,13.0,0.0,2.0


In [6]:
# On fait la différence entre la ligne n et la ligne n-1
spain_df['infected'] = spain_df['Confirmed'].diff()

In [7]:
# Définition du modèle SIR
beta,gamma = [0.01,0.1]

def SIR(t,y):
    S = y[0]
    I = y[1]
    R = y[2]
    return([-beta*S*I, beta*S*I-gamma*I, gamma*I])

In [8]:
# Résolution du modèle SIR
from scipy.integrate import solve_ivp

beta, gamma, alpha = [0.01, 0.1, 0.1]
solution_spain = solve_ivp(SIR, [0, 40], [51_470_000, 1, 0], t_eval=np.arange(0, 40, 1))

KeyboardInterrupt: 

In [ ]:
solution_spain

In [ ]:
def plot_epidemia(solution, infected, susceptible=False):
    fig = plt.figure(figsize=(12, 5))
    if susceptible:
        plt.plot(solution.t, solution.y[0])
    plt.plot(solution.t, solution.y[1])
    plt.plot(solution.t, solution.y[2])
    plt.plot(infected.reset_index(drop=True).index, infected, "k*:")
    plt.grid("True")
    if susceptible:
        plt.legend(["Susceptible", "Infected", "Recovered", "Original Data"])
    else:
        plt.legend(["Infected", "Recovered", "Original Data"])
    plt.show()

plot_epidemia(solution_spain, spain_df.loc[2:]['infected'])

In [ ]:
def sumsq_error(parameters):
    beta, gamma = parameters
    
    def SIR(t, y):
        S = y[0]
        I = y[1]
        R = y[2]
        return([-beta*S*I, beta*S*I-gamma*I, gamma*I])

    solution = solve_ivp(SIR, [0, nb_steps-1], [total_population, 1, 0], t_eval=np.arange(0, nb_steps, 1))
    
    return(sum((solution.y[1]-infected_population)**2))

In [ ]:
total_population = 51_470_000
infected_population = spain_df.loc[2:]['infected']
nb_steps = len(infected_population)

In [ ]:
%%time
from scipy.optimize import minimize

msol = minimize(sumsq_error, [0.001, 0.1], method='Nelder-Mead')
msol.x

In [ ]:
beta_optimal = msol.x[0]
gamma_optimal = msol.x[1]
print(beta_optimal)
print(gamma_optimal)

In [ ]:
beta = beta_optimal
gamma = gamma_optimal

def SIR(t, y):
    S = y[0]
    I = y[1]
    R = y[2]
    return([-beta*S*I, beta*S*I-gamma*I, gamma*I])

solution_spain_optimal = solve_ivp(SIR, [0, 40], [51_470_000*0.1, 1, 0], t_eval=np.arange(0, 40, 1))

In [ ]:
solution_spain_optimal